In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import nltk, warnings
import itertools
from statistics import mean
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import LocalOutlierFactor
from numpy import where
from google.colab import drive
#drive.mount('/content/drive')

In [10]:
#file which we got at the step 1
p1=pd.read_csv('customers_with_metrics(52&2).csv')
p1.head()


<ipython-input-10-1fcf0348cedc>:2: DtypeWarning: Columns (2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77) have mixed types. Specify dtype option on import or set low_memory=False.
  p1=pd.read_csv('customers_with_metrics(52&2).csv')


,Unnamed: 0,client_id,name1,Frequency1,Monetary1,name2,Frequency2,Monetary2,name3,Frequency3,...,Monetary49,name50,Frequency50,Monetary50,name51,Frequency51,Monetary51,name52,Frequency52,Monetary52
0,0,00125fe3b05ba65e08e6727837059786,sleeping,1.0,35.0000,sleeping,1.0,71.00000,sleeping,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sleeping,2.0,309.0
1,1,00157fd1a5d51498f4f6b163341ba789,sleeping,1.0,157.0000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,001ff3e3c3fe8ac01c5e8af8129e93fb,sleeping,2.0,114.0000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0025e578789f9f29a97b4f33c4edaa78,sleeping,2.0,1119.1276,NaN,NaN,NaN,loyal,5.0,...,2152.60444,loyal,4.0,2410.388,sleeping,2.0,785.0,sleeping,1.0,154.0
4,4,0035493aea0f02271c7204e02af85d6c,sleeping,3.0,338.9800,loyal,5.0,952.47644,loyal,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#create lists for using them later
l=[i for i in range(158)]
clusters=l[2:159:3]
freq=l[3:159:3]
mon=l[4:159:3]

In [13]:
for column in clusters:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna('churn')

#use the created lists to fill the dataset with zero values
for column in freq:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna(0)
for column in mon:
  p1[p1.columns[column]]=p1[p1.columns[column]].fillna(0)

In [15]:
p1.head()

,Unnamed: 0,client_id,name1,Frequency1,Monetary1,name2,Frequency2,Monetary2,name3,Frequency3,...,Monetary49,name50,Frequency50,Monetary50,name51,Frequency51,Monetary51,name52,Frequency52,Monetary52
0,0,00125fe3b05ba65e08e6727837059786,sleeping,1.0,35.0000,sleeping,1.0,71.00000,sleeping,3.0,...,0.00000,churn,0.0,0.000,churn,0.0,0.0,sleeping,2.0,309.0
1,1,00157fd1a5d51498f4f6b163341ba789,sleeping,1.0,157.0000,churn,0.0,0.00000,churn,0.0,...,0.00000,churn,0.0,0.000,churn,0.0,0.0,churn,0.0,0.0
2,2,001ff3e3c3fe8ac01c5e8af8129e93fb,sleeping,2.0,114.0000,churn,0.0,0.00000,churn,0.0,...,0.00000,churn,0.0,0.000,churn,0.0,0.0,churn,0.0,0.0
3,3,0025e578789f9f29a97b4f33c4edaa78,sleeping,2.0,1119.1276,churn,0.0,0.00000,loyal,5.0,...,2152.60444,loyal,4.0,2410.388,sleeping,2.0,785.0,sleeping,1.0,154.0
4,4,0035493aea0f02271c7204e02af85d6c,sleeping,3.0,338.9800,loyal,5.0,952.47644,loyal,5.0,...,0.00000,churn,0.0,0.000,churn,0.0,0.0,churn,0.0,0.0


In [17]:
#save correct dataset (with zeros)
p1.drop(columns=['Unnamed: 0'], inplace=True)
p1.to_csv('customers_with_metrics_52_0.csv')

#### Keep only cluster values to calculate the intensity of flows

In [19]:
freq=l[2:159:3]
mon=l[3:159:3]
df = p1.drop(p1.columns[freq+mon], axis=1)

In [20]:
df.head()

,client_id,name1,name2,name3,name4,name5,name6,name7,name8,name9,...,name43,name44,name45,name46,name47,name48,name49,name50,name51,name52
0,00125fe3b05ba65e08e6727837059786,sleeping,sleeping,sleeping,sleeping,sleeping,churn,churn,churn,churn,...,churn,churn,churn,churn,churn,churn,churn,churn,churn,sleeping
1,00157fd1a5d51498f4f6b163341ba789,sleeping,churn,churn,churn,churn,churn,churn,churn,churn,...,churn,churn,churn,churn,churn,churn,churn,churn,churn,churn
2,001ff3e3c3fe8ac01c5e8af8129e93fb,sleeping,churn,churn,churn,churn,churn,churn,churn,churn,...,sleeping,churn,churn,churn,sleeping,churn,churn,churn,churn,churn
3,0025e578789f9f29a97b4f33c4edaa78,sleeping,churn,loyal,sleeping,churn,sleeping,sleeping,churn,churn,...,sleeping,loyal,sleeping,sleeping,loyal,sleeping,loyal,loyal,sleeping,sleeping
4,0035493aea0f02271c7204e02af85d6c,sleeping,loyal,loyal,loyal,loyal,sleeping,loyal,loyal,sleeping,...,churn,sleeping,churn,churn,churn,churn,churn,churn,churn,churn


In [21]:
#change names to the correct ones
df_r=df.replace('churn','4: churn').replace('champions','3: champions').replace('loyal','2: loyal').replace('sleeping','1: sleeping')
df_r.columns=df_r.columns.str.replace('name','period')
df_r.head()

,client_id,period1,period2,period3,period4,period5,period6,period7,period8,period9,...,period43,period44,period45,period46,period47,period48,period49,period50,period51,period52
0,00125fe3b05ba65e08e6727837059786,1: sleeping,1: sleeping,1: sleeping,1: sleeping,1: sleeping,4: churn,4: churn,4: churn,4: churn,...,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,1: sleeping
1,00157fd1a5d51498f4f6b163341ba789,1: sleeping,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,...,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn
2,001ff3e3c3fe8ac01c5e8af8129e93fb,1: sleeping,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,...,1: sleeping,4: churn,4: churn,4: churn,1: sleeping,4: churn,4: churn,4: churn,4: churn,4: churn
3,0025e578789f9f29a97b4f33c4edaa78,1: sleeping,4: churn,2: loyal,1: sleeping,4: churn,1: sleeping,1: sleeping,4: churn,4: churn,...,1: sleeping,2: loyal,1: sleeping,1: sleeping,2: loyal,1: sleeping,2: loyal,2: loyal,1: sleeping,1: sleeping
4,0035493aea0f02271c7204e02af85d6c,1: sleeping,2: loyal,2: loyal,2: loyal,2: loyal,1: sleeping,2: loyal,2: loyal,1: sleeping,...,4: churn,1: sleeping,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn,4: churn


In [22]:
#calculate the number of customers in each cluster in each month
a=df_r[['period1', 'client_id']].groupby('period1').count()
a1=a.reset_index()
a1.columns=['column', 'count1']
for column in df_r.columns.to_list()[2:]:
  aa=df_r[[column, 'client_id']].groupby(column).count()
  aaa=aa.reset_index()
  aaa.columns=['column', column]
  a1=a1.merge(aaa, on='column', how='left')
a1

,column,count1,period2,period3,period4,period5,period6,period7,period8,period9,...,period43,period44,period45,period46,period47,period48,period49,period50,period51,period52
0,1: sleeping,5716,5814,5735,6029,5429,5700,5505,5680,5216,...,5092,5015,5066,5066,5018,4962,4661,4898,4964,5058
1,2: loyal,478,448,447,477,887,421,424,415,865,...,365,379,403,382,380,371,752,765,395,820
2,3: champions,1094,1053,1020,963,845,1005,981,996,804,...,908,1082,966,1040,1068,986,773,880,1031,907
3,4: churn,24333,24306,24419,24152,24460,24495,24711,24530,24736,...,25256,25145,25186,25133,25155,25302,25435,25078,25231,24836


In [27]:
a2=a1.T
a2=a2.iloc[1:,0:]
a2.columns=['sleeping', 'loyal',	'champions', 'churn']
a2.head()

,sleeping,loyal,champions,churn
count1,5716,478,1094,24333
period2,5814,448,1053,24306
period3,5735,447,1020,24419
period4,6029,477,963,24152
period5,5429,887,845,24460


In [28]:
#save this statistics
a2.to_csv('count_of_clusters_52.csv')

In [29]:
#create dataset to save number of flows
df=pd.DataFrame(columns=['sleeping_to_sleeping','sleeping_to_loyal','sleeping_to_champions','sleeping_to_churn','loyal_to_sleeping','loyal_to_loyal','loyal_to_champions','loyal_to_churn','champions_to_sleeping','champions_to_loyal','champions_to_champions','champions_to_churn','churn_to_sleeping','churn_to_loyal','churn_to_champions','churn_to_churn'])

In [30]:
#code to calculate the number of customers in each flow
all=[]
for stroka in [0,1,2,3]:
  col=0
  spisok=[]
  for period in range(1,24):

    df2=df_r[['client_id', f'period{str(period)}', f'period{str(period+1)}']]
    output = df2.pivot_table(index=f'period{str(period+1)}', columns=f'period{str(period)}', values='client_id', aggfunc='count', margins=True,margins_name='Grand Total')

    #it is necessary to change the numbers in range(...): 0,1 then 1,2 then 2,3 and finally 3,4
    for i in range(3,4):
      #output.iloc[int(i),:]=round((output.iloc[int(i),:]/output.iloc[4,:])*100,1)
      outputt=output.iloc[:4, :4]
    spisok.append(outputt.iloc[stroka,int(i)])

  #and it is necessary to change columns in dataset: 0:4 then 4:8 then 8:12 then 12:16
  df[df.iloc[:,12:16].columns[int(stroka)]]=spisok
print(spisok)

[20462, 20539, 20473, 20494, 20662, 20907, 20960, 20951, 20655, 20436, 20609, 20402, 20088, 19563, 18943, 17610, 17663, 19767, 20399, 20569, 20707, 20704, 20388]


In [ ]:
#final dataset
df.to_csv('count_of_flows.csv')